In [5]:
import pandas as pd
import numpy as np
import haversine as hs
import math

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [6]:
centroids=pd.read_csv("./statsnzstatistical-area-2-2020-centroid-true-CSV/statistical-area-2-2020-centroid-true.csv")

In [7]:
def spelling(df):
    df.loc[df['Area']=="Putararu Rural",'Area']="Putaruru Rural"
    df.loc[df['Area']=="Putararu",'Area']="Putaruru"
    df.loc[df['Area']=="Whangaparoa Central",'Area']="Whangaparaoa Central"
    df.loc[df['Area']=="Oceanic Manawatu-Wanganui Region East",'Area']="Oceanic Manawatu-Whanganui Region East"
    df.loc[df['Area']=="Oceanic Manawatu-Wanganui Region West",'Area']="Oceanic Manawatu-Whanganui Region West"
    return(df)

def dropAreas(df):
#This function is necessary when working with the csv geom file, not the shapefile
    df = df[df.Area != 'Antipodes Islands']
    df = df[df.Area != 'Auckland Islands']
    df = df[df.Area != 'Bounty Islands']
    df = df[df.Area != 'Campbell Island']
    df = df[df.Area != 'Kermadec Islands']
    df = df[df.Area != 'New Zealand Economic Zone']
    df = df[df.Area != 'Oceanic Antipodes Islands']
    df = df[df.Area != 'Oceanic Auckland Islands']
    df = df[df.Area != 'Oceanic Bounty Islands']
    df = df[df.Area != 'Oceanic Campbell Island']
    df = df[df.Area != 'Oceanic Kermadec Islands']
    df = df[df.Area != 'Oceanic Oil Rig Southland']
    df = df[df.Area != 'Oceanic Oil Rig Taranaki']
    df = df[df.Area != 'Oceanic Snares Islands']
    df = df[df.Area != 'Ross Dependency']
    df = df[df.Area != 'Snares Islands']
    return(df)

def addAreas(group):
    rows_list=[]
    if group == "A":
        for year in [2000,2020]:
            dict1={'Area':'Te Kauwhata West', 'TotInd_GeogUnits':np.nan, 'TotInd_EmpCo':np.nan, 'F_GeogUnits':np.nan,
                   'F_EmpCo':np.nan,'G_GeogUnits':np.nan, 'G_EmpCo':np.nan, 'I461_GeogUnits':np.nan, 'I461_EmpCo':np.nan,
                   'I471_GeogUnits':np.nan, 'I471_EmpCo':np.nan, 'I481_GeogUnits':np.nan, 'I481_EmpCo':np.nan,
                   'I51_GeogUnits':np.nan, 'I51_EmpCo':np.nan, 'I53_GeogUnits':np.nan, 'I53_EmpCo':np.nan,
                   'ParentArea':'WaikatoRegion', 'Year':year}
            rows_list.append(dict1)
            dict1={'Area':'Whale Bay', 'TotInd_GeogUnits':np.nan, 'TotInd_EmpCo':np.nan, 'F_GeogUnits':np.nan,
                   'F_EmpCo':np.nan,'G_GeogUnits':np.nan, 'G_EmpCo':np.nan, 'I461_GeogUnits':np.nan, 'I461_EmpCo':np.nan,
                   'I471_GeogUnits':np.nan, 'I471_EmpCo':np.nan, 'I481_GeogUnits':np.nan, 'I481_EmpCo':np.nan,
                   'I51_GeogUnits':np.nan, 'I51_EmpCo':np.nan, 'I53_GeogUnits':np.nan, 'I53_EmpCo':np.nan,
                   'ParentArea':'WaikatoRegion', 'Year':year}
            rows_list.append(dict1)
    return(rows_list)

In [8]:
loc1=(-37.149892,174.632167)
loc2=(-38.122698,174.798582)
hs.haversine(loc1,loc2)

109.1592260202929

In [9]:
#Read in the complete set
groupA=pd.read_csv("../CompleteSet_GroupA.csv")
groupA=groupA.drop("Unnamed: 0",axis=1)


#Strip all leading whitespace in Area column
groupA['Area'] = groupA['Area'].apply(lambda x: x.strip())
groupA=groupA[groupA.ParentArea!='NewZealand']
groupA=groupA[groupA.ParentArea!='TOTAL']
groupA=spelling(groupA)
additional=pd.DataFrame(addAreas("A"))
groupA = pd.concat([groupA, additional])
groupA = dropAreas(groupA)

groupA=groupA.fillna(0)



In [10]:
# For each region, calculate the weighted centroid in 2000 and 2020

regionMaster=['AreaOutsideRegion','AucklandRegion','BayOfPlentyRegion','CanterburyRegion','GisborneRegion','HawkesBayRegion',
             'Manawatu-WanganuiRegion','MarlboroughRegion','NelsonRegion','NorthlandRegion','OtagoRegion','SouthlandRegion',
              'TaranakiRegion','TasmanRegion','WaikatoRegion','WellingtonRegion','WestCoastRegion']
sectorsMaster=["ALL","RETAIL","WHOLESALE","TRANS_STOR","COMBO"]

In [11]:
groupA.columns

Index(['Area', 'TotInd_GeogUnits', 'TotInd_EmpCo', 'F_GeogUnits', 'F_EmpCo',
       'G_GeogUnits', 'G_EmpCo', 'I461_GeogUnits', 'I461_EmpCo',
       'I471_GeogUnits', 'I471_EmpCo', 'I481_GeogUnits', 'I481_EmpCo',
       'I51_GeogUnits', 'I51_EmpCo', 'I53_GeogUnits', 'I53_EmpCo',
       'ParentArea', 'Year'],
      dtype='object')

In [12]:
def calcCentroid(df,region,year,sector):
    df=df[df.Year==year]
    df=df[df.ParentArea==region]
    x_cum=0
    y_cum=0
    fac_cum=0
    for i in range(len(df)):
        record=centroids.loc[centroids["SA22020_V1_00_NAME_ASCII"]==df.iloc[i,0],]
        lat=record.iloc[0,6]
        lon=record.iloc[0,7]
        if sector=="ALL":
            fac_num=df.iloc[i,1]
        elif sector=="RETAIL":
            fac_num=df.iloc[i,5]
        elif sector=="WHOLESALE":
            fac_num=df.iloc[i,3]
        elif sector=="TRANS_STOR":
            fac_num=df.iloc[i,7]+df.iloc[i,9]+df.iloc[i,11]+df.iloc[i,13]+df.iloc[i,15] 
        elif sector=="COMBO":
            fac_num=df.iloc[i,7]+df.iloc[i,9]+df.iloc[i,11]+df.iloc[i,13]+df.iloc[i,15]+df.iloc[i,3]+df.iloc[i,5]
        x_cum=x_cum+lat*fac_num
        y_cum=y_cum+lon*fac_num
        fac_cum=fac_cum+fac_num
    if fac_cum>0:
        cent=[x_cum/fac_cum,y_cum/fac_cum]
    else:
        cent=[np.nan,np.nan]
    return(cent)       

In [ ]:
## 2000 All industries

regions_list=[]

year=2000
for sector in sectorsMaster:
    for region in regionMaster:
        centr=calcCentroid(groupA,region,year,sector)
        dict1={'Region':region, 'Year':year, 'Sectors':sector, 'Centroid_lat':centr[0],'Centroid_lon':centr[1]}
        regions_list.append(dict1)
centroid_df = pd.DataFrame(regions_list)

In [10]:
## 2020 All industries

regions_list=[]

year=2020
for sector in sectorsMaster:
    for region in regionMaster:
        centr=calcCentroid(groupA,region,year,sector)
        dict1={'Region':region, 'Year':year, 'Sectors':sector, 'Centroid_lat':centr[0],'Centroid_lon':centr[1]}
        regions_list.append(dict1)
centroid_df = pd.concat([centroid_df,pd.DataFrame(regions_list)])

In [11]:
centroid_df.head()

,Region,Year,Sectors,Centroid_lat,Centroid_lon
0,AreaOutsideRegion,2000,ALL,-43.917364,-176.515085
1,AucklandRegion,2000,ALL,-36.866076,174.771947
2,BayOfPlentyRegion,2000,ALL,-37.858811,176.330791
3,CanterburyRegion,2000,ALL,-43.614256,172.337225
4,GisborneRegion,2000,ALL,-38.549210,177.971270


In [13]:
haver_list=[]
for sector in sectorsMaster:
    for region in regionMaster:
        record1=centroid_df.loc[(centroid_df['Region']==region)&(centroid_df['Year']==2000)&(centroid_df['Sectors']==sector),]
        record2=centroid_df.loc[(centroid_df['Region']==region)&(centroid_df['Year']==2020)&(centroid_df['Sectors']==sector),]
        loc1=(record1.iloc[0,3],record1.iloc[0,4])
        loc2=(record2.iloc[0,3],record2.iloc[0,4])
        dict1={'Region':region, 'Sector':sector, 'Shift_lat':f'{(record2.iloc[0,3]-record1.iloc[0,3]):.20f}', 'Shift_lon':f'{(record2.iloc[0,4]-record1.iloc[0,4]):.20f}', 'Haversine':f'{hs.haversine(loc1,loc2):.20f}'}
        haver_list.append(dict1)
haver_df=pd.DataFrame(haver_list)

In [14]:
haver_df

,Region,Sector,Shift_lat,Shift_lon,Haversine
0,AreaOutsideRegion,ALL,-0.00000000000001421085,0.00000000000002842171,0.00000000000248090899
1,AucklandRegion,ALL,0.00387081983935644303,0.00143372397201346757,0.44891707960250226694
2,BayOfPlentyRegion,ALL,0.03425641808252777309,-0.03150669122746307949,4.70786366277794066093
3,CanterburyRegion,ALL,0.01405984635508161773,0.02545376343894645288,2.57763875854604540550
4,GisborneRegion,ALL,-0.02672821850672590926,0.00147228464365412037,2.97480190608194794422
5,HawkesBayRegion,ALL,-0.00643565344747543122,0.00395873137148328169,0.79194426419281294116
6,Manawatu-WanganuiRegion,ALL,-0.01052323224402584856,-0.00969716646983442843,1.43127633088732575395
7,MarlboroughRegion,ALL,-0.01875643656027392581,-0.00596851232342032745,2.14402113217390688504
8,NelsonRegion,ALL,-0.00436372013218999655,-0.00344801023626928327,0.56431582686365466817
9,NorthlandRegion,ALL,-0.00783409563817372145,0.03809891560692335588,3.55349565879335260732


## Average distance to centroid

In [101]:
def avgDistToCent(df,region,year,sector,cent_df):
    df=df[df.Year==year]
    df=df[df.ParentArea==region]
    #fetch centroid
    cent_record=cent_df.loc[(cent_df['Region']==region)&(cent_df['Sectors']==sector)&(cent_df['Year']==year),]
    centroid=[cent_record.iloc[0,3],cent_record.iloc[0,4]]
    haver_cum=0
    fac_cum=0
    for i in range(len(df)):
        if sector=="ALL":
            fac_num=df.iloc[i,1]
        elif sector=="RETAIL":
            fac_num=df.iloc[i,5]
        elif sector=="WHOLESALE":
            fac_num=df.iloc[i,3]
        elif sector=="TRANS_STOR":
            fac_num=df.iloc[i,7]+df.iloc[i,9]+df.iloc[i,11]+df.iloc[i,13]+df.iloc[i,15] 
        elif sector=="COMBO":
            fac_num=df.iloc[i,7]+df.iloc[i,9]+df.iloc[i,11]+df.iloc[i,13]+df.iloc[i,15]+df.iloc[i,3]+df.iloc[i,5]
        current=centroids.loc[centroids["SA22020_V1_00_NAME_ASCII"]==df.iloc[i,0],]
        current_loc=[current.iloc[0,6],current.iloc[0,7]]
        haver=hs.haversine(centroid,current_loc)
        haver_cum=haver_cum+haver*fac_num
        fac_cum=fac_cum+fac_num
    if fac_cum>0:
        avgDist=haver_cum/fac_cum
    else:
        avgDist=np.nan
    return(avgDist)
 

In [107]:

regions_list=[]
for sector in sectorsMaster:
    for region in regionMaster:
        avgDist2000=avgDistToCent(groupA,region,2000,sector,centroid_df)
        avgDist2020=avgDistToCent(groupA,region,2020,sector,centroid_df)
        dict1={'Region':region, 'Year':year, 'Sectors':sector, 'AvgDistToCent_2000':f'{avgDist2000:.20f}','AvgDistToCent_2020':f'{avgDist2020:.20f}','Diff':f'{(avgDist2020-avgDist2000):.20f}'}
        regions_list.append(dict1)
distToCent_df = pd.DataFrame(regions_list)

In [108]:
distToCent_df

,Region,Year,Sectors,AvgDistToCent_2000,AvgDistToCent_2020,Diff
0,AreaOutsideRegion,2000,ALL,0.00000000000215731023,0.00000000000070732407,-0.00000000000144998616
1,AucklandRegion,2000,ALL,13.02053093601967859172,12.99872845020885847589,-0.02180248581082011583
2,BayOfPlentyRegion,2000,ALL,33.30320164385869929902,29.84376091318420520793,-3.45944073067449409109
3,CanterburyRegion,2000,ALL,47.58180447737968421507,42.78359633931444250265,-4.79820813806524171241
4,GisborneRegion,2000,ALL,21.76520044456270142064,18.17420734448522878779,-3.59099310007747263285
5,HawkesBayRegion,2000,ALL,22.51853962449329316087,20.36794118394513830594,-2.15059844054815485492
6,Manawatu-WanganuiRegion,2000,ALL,43.26923766079220001757,41.37258211721371026215,-1.89665554357848975542
7,MarlboroughRegion,2000,ALL,21.55761175892052250447,21.54975671933112835177,-0.00785503958939415270
8,NelsonRegion,2000,ALL,2.93490048549529358723,3.22799489174291176496,0.29309440624761817773
9,NorthlandRegion,2000,ALL,43.94133408021763642637,43.54525785409271065873,-0.39607622612492576764


In [109]:
distToCent_df.to_csv('DistanceToCentroids.csv')
haver_df.to_csv("CentroidShift.csv")
centroid_df.to_csv('WeightedCentroids.csv')

In [15]:
haver_df.to_csv("CentroidShift.csv")